In [42]:
%matplotlib inline
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
import pandas as pd
import os
import warnings
warnings.simplefilter('ignore')
import numpy as np

In [88]:
rent = "final.csv"
rent_df = pd.read_csv(rent)
rent_df = rent_df.drop(["Unnamed: 0"], axis = 1)
rent_df.head()

,Property_Code,Tenant_Code,Event,DateOccurred,Unit_Code,DateMoveIn,DateMoveOut,Rent,Vacancy,SQFT,BedRooms,Bathroom,chrent,LengthStay
0,ce,t0013745,Rent Change,4/30/2013,C05266,11/9/2006,NaN,870.0,0,1400,3,2.5,870.0,NaN
1,ce,t0013745,Rent Change,5/31/2014,C05266,11/9/2006,1/31/2015,896.0,1,1400,3,2.5,26.0,245 days 00:00:00.000000000
2,ce,t0013747,Rent Change,4/30/2013,C05270,7/16/2012,8/11/2013,770.0,1,1088,2,1.5,770.0,103 days 00:00:00.000000000
3,ce,t0013748,Rent Change,4/30/2013,C05272,9/28/2012,NaN,770.0,0,1088,2,1.5,770.0,NaN
4,ce,t0013748,Rent Change,9/30/2013,C05272,9/28/2012,NaN,800.0,0,1088,2,1.5,30.0,NaN


In [92]:
sqft_bins = [0, 600, 1000, 2000]
bin_names = ["small", "medium", "large"]
rent_df["Apt Size"] = pd.cut(rent_df["SQFT"], sqft_bins, labels = bin_names)
rent_df.loc[rent_df["Vacancy"] == 1, "Vacancy"] = "Vacant"
rent_df.loc[rent_df["Vacancy"] == 0, "Vacancy"] = "Occupied"
rent_df

,Property_Code,Tenant_Code,Event,DateOccurred,Unit_Code,DateMoveIn,DateMoveOut,Rent,Vacancy,SQFT,BedRooms,Bathroom,chrent,LengthStay,Apt Size
0,ce,t0013745,Rent Change,4/30/2013,C05266,11/9/2006,NaN,870.0,Occupied,1400,3,2.5,870.0,NaN,large
1,ce,t0013745,Rent Change,5/31/2014,C05266,11/9/2006,1/31/2015,896.0,Vacant,1400,3,2.5,26.0,245 days 00:00:00.000000000,large
2,ce,t0013747,Rent Change,4/30/2013,C05270,7/16/2012,8/11/2013,770.0,Vacant,1088,2,1.5,770.0,103 days 00:00:00.000000000,large
3,ce,t0013748,Rent Change,4/30/2013,C05272,9/28/2012,NaN,770.0,Occupied,1088,2,1.5,770.0,NaN,large
4,ce,t0013748,Rent Change,9/30/2013,C05272,9/28/2012,NaN,800.0,Occupied,1088,2,1.5,30.0,NaN,large
5,ce,t0013748,Rent Change,11/1/2014,C05272,9/28/2012,10/31/2015,824.0,Vacant,1088,2,1.5,24.0,364 days 00:00:00.000000000,large
6,ce,t0013749,Rent Change,4/30/2013,C05274,11/26/2008,NaN,880.0,Occupied,1400,3,2.5,880.0,NaN,large
7,ce,t0013749,Rent Change,10/31/2013,C05274,11/26/2008,NaN,1795.0,Occupied,1400,3,2.5,915.0,NaN,large
8,ce,t0013749,Rent Change,12/1/2013,C05274,11/26/2008,NaN,880.0,Occupied,1400,3,2.5,-915.0,NaN,large
9,ce,t0013749,Rent Change,12/31/2013,C05274,11/26/2008,NaN,915.0,Occupied,1400,3,2.5,35.0,NaN,large


In [93]:
X = rent_df[["Apt Size", "BedRooms", "Bathroom", "chrent", "Property_Code"]]
y = rent_df["Vacancy"]
print(X.shape, y.shape)

(6654, 5) (6654,)


In [94]:
data = X.copy()

data_binary_encoded = pd.get_dummies(data, columns = ["Apt Size", "Property_Code"])
data_binary_encoded.head()

,BedRooms,Bathroom,chrent,Apt Size_small,Apt Size_medium,Apt Size_large,Property_Code_ce,Property_Code_fh,Property_Code_mpg,Property_Code_sh
0,3,2.5,870.0,0,0,1,1,0,0,0
1,3,2.5,26.0,0,0,1,1,0,0,0
2,2,1.5,770.0,0,0,1,1,0,0,0
3,2,1.5,770.0,0,0,1,1,0,0,0
4,2,1.5,30.0,0,0,1,1,0,0,0


In [95]:
data_binary_encoded = data_binary_encoded.fillna(0)

In [96]:
data_binary_encoded.dtypes

BedRooms               int64
Bathroom             float64
chrent               float64
Apt Size_small         uint8
Apt Size_medium        uint8
Apt Size_large         uint8
Property_Code_ce       uint8
Property_Code_fh       uint8
Property_Code_mpg      uint8
Property_Code_sh       uint8
dtype: object

In [97]:
data_binary_encoded ["BedRooms"]= data_binary_encoded["BedRooms"].astype(float)
data_binary_encoded ["Apt Size_small"]= data_binary_encoded["Apt Size_small"].astype(float)
data_binary_encoded ["Apt Size_medium"]= data_binary_encoded["Apt Size_medium"].astype(float)
data_binary_encoded ["Apt Size_large"]= data_binary_encoded["Apt Size_large"].astype(float)
data_binary_encoded ["Property_Code_ce"]= data_binary_encoded["Property_Code_ce"].astype(float)
data_binary_encoded ["Property_Code_fh"]= data_binary_encoded["Property_Code_fh"].astype(float)
data_binary_encoded ["Property_Code_mpg"]= data_binary_encoded["Property_Code_mpg"].astype(float)
data_binary_encoded ["Property_Code_sh"]= data_binary_encoded["Property_Code_sh"].astype(float)

In [98]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_binary_encoded, y, stratify=y, random_state=42)

In [99]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [100]:
classifier.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [101]:
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.7472945891783567
Testing Data Score: 0.7469951923076923


In [102]:
predictions = classifier.predict(X_test)
print(f"First 10 Predictions:   {predictions[:10]}")
print(f"First 10 Actual labels: {y_test[:10].tolist()}")

First 10 Predictions:   ['Occupied' 'Occupied' 'Occupied' 'Occupied' 'Occupied' 'Occupied'
 'Occupied' 'Occupied' 'Occupied' 'Occupied']
First 10 Actual labels: ['Occupied', 'Vacant', 'Occupied', 'Vacant', 'Occupied', 'Occupied', 'Vacant', 'Occupied', 'Occupied', 'Vacant']


In [103]:
X_test.head()

,BedRooms,Bathroom,chrent,Apt Size_small,Apt Size_medium,Apt Size_large,Property_Code_ce,Property_Code_fh,Property_Code_mpg,Property_Code_sh
702,2.0,1.0,20.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2778,2.0,1.5,8.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3023,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
5294,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2586,1.0,1.0,7.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0


In [104]:
pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)

,Prediction,Actual
0,Occupied,Occupied
1,Occupied,Vacant
2,Occupied,Occupied
3,Occupied,Vacant
4,Occupied,Occupied
5,Occupied,Occupied
6,Occupied,Vacant
7,Occupied,Occupied
8,Occupied,Occupied
9,Occupied,Vacant
